# Does SSA (Sistem Satu Arah - Unidirectional Road System) implementation on some road segments in Depok city make any improvement to the traffic condition of the city?

- Comparing the traffic conditions (in terms of travel time) between various locations in Depok city before (< July 2017) and after (> July 2017) the implementation of SSA
- Using HERE REST API to derive past traffic conditions

### Limitations
Apparently HERE REST API only provides precise historical travel times from 1 January 2019 onward. Travel times before that time are averaged out and categorised into 'morning travel' and 'evening travel'.

# information input

In [19]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import urllib.request
import sys
import osmnx as ox
from shapely.geometry import Point, Polygon
import gmplot
from utils import *

app_code = ''
app_id = ''


In [2]:
#list koordinat
points = [Point(106.831913, -6.377133), #01 pertigaan juanda-margonda arah ramanda
          Point(106.825930, -6.389564), #02 pertigaan ramanda arah terminal depok
          Point(106.825812, -6.389415), #03 pertigaan ramanda arah depok mall
          Point(106.814649, -6.392354), #04 pasar baru jln nusantara
          Point(106.812714, -6.399052), #05 jln raya sawangan
          Point(106.813877, -6.397228), #06 rumah (sarikaya)
          Point(106.813627, -6.399058), #07 dewi sartika, perlimaan, lajur lama
          Point(106.813580, -6.399127), #08 dewi sartika, perlimaan, lajur baru
          Point(106.818997, -6.399890), #09 jln kartini - dewi sartika, arah margonda
          Point(106.819098, -6.399924), #10 jln kartini - dewi sartika, arah citayam
          Point(106.823846, -6.393196), #11 ITC depok
          Point(106.824034, -6.393209), #12 sebrang ITC depok
         ]

In [3]:
points_name = ['01 pertigaan juanda-margonda arah ramanda',
          '02 pertigaan ramanda arah terminal depok',
          '03 pertigaan ramanda arah depok mall',
          '04 pasar baru jln nusantara',
          '05 jln raya sawangan',
          '06 rumah (sarikaya)',
          '07 dewi sartika, perlimaan, lajur lama',
          '08 dewi sartika, perlimaan, lajur baru',
          '09 jln kartini - dewi sartika, arah margonda',
          '10 jln kartini - dewi sartika, arah citayam',
          '11 ITC depok',
          '12 sebrang ITC depok']

In [4]:
times2019_mon = ['2019-02-04T{}:00:00'.format(x) for x in ['07', '08', '09', '17', '18', '19']]
times2019_wed = ['2019-02-06T{}:00:00'.format(x) for x in ['07', '08', '09', '17', '18', '19']]
times2019_fri = ['2019-02-08T{}:00:00'.format(x) for x in ['07', '08', '09', '17', '18', '19']]
times2019_sat = ['2019-02-09T{}:00:00'.format(x) for x in ['07', '08', '09', '17', '18', '19']]

In [5]:
times2019_fri

['2019-02-08T07:00:00',
 '2019-02-08T08:00:00',
 '2019-02-08T09:00:00',
 '2019-02-08T17:00:00',
 '2019-02-08T18:00:00',
 '2019-02-08T19:00:00']

# CSV generation

In [8]:
times = [times2019_mon, times2019_wed, times2019_fri, times2019_sat]

In [16]:
gdf_master = gpd.GeoDataFrame()

for tms in times:
    for p1 in range(12): #for each starting position
        print(p1)
        for p2 in range(12): #for each ending position
            if p1 != p2:
                start = points[p1]
                end = points[p2]

                #2019 data
                for t in range(6): #for each time of the day
                    dep_hour = tms[t]
                    if t==0:
                        gdf = get_df(start=start, end=end, time=dep_hour, app_code=app_code, app_id=app_id)
                        gdf['clock_time'] = dep_hour
                    else:
                        gdf2 = get_df(start=start, end=end, time=dep_hour, app_code=app_code, app_id=app_id)
                        gdf2['clock_time'] = dep_hour
                        gdf = gdf.append(gdf2)
                gdf['From'] = points_name[p1]
                gdf['To'] = points_name[p2]
                gdf_master = gdf_master.append(gdf)

0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11


In [18]:
gdf_master.to_csv('data_2019.csv')